<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
#!/usr/bin/env python
# coding: utf-8

# <h1>Table of Contents<span class="tocSkip"></span></h1>
# <div class="toc"><ul class="toc-item"></ul></div>

import sys
import torch, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM  # === (изменено) убран pipeline, чтобы гарантированно не вызывался

import pandas as pd
from IPython.display import display

import time as tm
t1_itogo = tm.time()

pd.set_option("display.max_rows", None) #убрали ограничение на число строк при отображении датафрейма
# ограничение на 20 строк
# pd.set_option("display.max_rows", 20)
from pathlib import Path




# убрала все предупреждения, потому что не смоогла вырубить предупреждения для модификации summary для коротких промптов
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# === BUILD STAMP (изменено) ===
print("BUILD: Qwen3 switch, SUMMARY hard-off | 2025-08-06")
try:
    import transformers, sys
    print("versions:", "torch=", torch.__version__, "transformers=", transformers.__version__, "python=", sys.version.split()[0])
except Exception as _e:
    print("version print failed:", _e)

if 'pipeline' in globals():
    print("⚠️ WARNING: found 'pipeline' symbol in globals(); ensure no 'from transformers import pipeline' left.")

"""
Скрипт сравнения распределений P(next_token) с поддержкой:
----------------------------------------------------------------
1. **Внутренних промптов** (список в коде) **или**
2. **Промптов из текстового файла** в той же директории.

Флаг `USE_FILE_PROMPTS` выбирает источник.
Если `True`, файл `PROMPT_FILE` читается целиком и используется как **один** длинный prompt.

Модификация «summary + слова» печатает краткое summary исходного prompt‑а.
Добавлена **безопасная токенизация**: если длиннее лимита модели (1024 для GPT‑2),
текст автоматически усечётся, а в консоль выведется предупреждение.
""";

C:\Users\cab\anaconda3\envs\torch241\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BUILD: Qwen3 switch, SUMMARY hard-off | 2025-08-06
versions: torch= 2.6.0+cpu transformers= 4.55.0 python= 3.10.18


In [2]:
# ---------- ПАРАМЕТРЫ ----------
# === QWEN3 SWITCH  ===
MODEL_NAME   = "Qwen/Qwen3-0.6B-Base"  # публичный идентификатор на HF; базовая (не чат) версия  # <-- укажите точный ID вашей Qwen3 модели, если другой (напр., локальное имя/путь)
TRUST_REMOTE_CODE = True      # для семейств Qwen обычно требуется
STEPS        = 50
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"

# Флаг, управляющий включением суммаризации как отдельной модификации.
# === QWEN3 SWITCH (изменено) === — теперь реально управляет созданием пайплайна
# === (изменено пользователем) — summary временно отключён
SUMMARY_ENABLED = False

# Суммаризация
SUMM_MODEL   = "sshleifer/distilbart-cnn-12-6"
CUSTOM_WORDS = "Сводка:"

# Выбор источника промптов
USE_FILE_PROMPTS = True
PROMPT_FILE      = "prompt.txt"


In [3]:
# ---------- 1. Модели ----------
# (изменено) — для Qwen добавлен trust_remote_code
_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=TRUST_REMOTE_CODE)
# (изменено) — для Qwen добавлен trust_remote_code
_model     = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=TRUST_REMOTE_CODE).to(DEVICE).eval()

# === DISABLE SUMMARY HARD (изменено) ===
# Полностью отключаем создание summarizer: никакого pipeline() вообще
_summarizer = None
if SUMMARY_ENABLED:
    print("⚠️ SUMMARY_ENABLED=True, но summarizer временно принудительно отключён."
          "Переключите SUMMARY_ENABLED=False или обновите torch>=2.6/выберите safetensors-модель.")
    SUMMARY_ENABLED = False

# (изменено) — более надёжное определение лимита контекста для разных архитектур
CTX_LIMIT = (
    getattr(_model.config, "max_position_embeddings", None)
    or getattr(_model.config, "n_positions", None)
    or getattr(_tokenizer, "model_max_length", 1024)
)

In [4]:
# ---------- 2. Вспомогательные функции ----------
def _next_token_probs(ids):
    with torch.no_grad():
        logits = _model(ids).logits[0, -1]
    return torch.softmax(logits, dim=-1).cpu()

def _kl(p, q, eps=0):  # нормально работает при eps=1e-8
    p = p + eps
    q = q + eps
    return torch.sum(p * torch.log(p / q)).item()

def _cos(p, q):
    return F.cosine_similarity(p, q, dim=0).item()

def _multi_step(prompt: str, n: int):
    token_ids = _tokenizer.encode(prompt)
    if len(token_ids) > CTX_LIMIT:
        print(f"⚠️ Промпт содержит {len(token_ids)} токенов и будет усечён до {CTX_LIMIT}.")
    ids = _tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=CTX_LIMIT
    )["input_ids"].to(DEVICE)

    dists = []
    for _ in range(n):
        probs = _next_token_probs(ids)
        dists.append(probs)
        next_id = probs.argmax().unsqueeze(0).unsqueeze(0).to(DEVICE)
        ids = torch.cat([ids, next_id], dim=1)
    return dists


In [5]:
# ---------- 3. Источник промптов и модификаций ----------

USE_FILE_PROMPTS = False           # или False
PROMPT_FILE = "prompt.txt"        # имя файла

if USE_FILE_PROMPTS:
    path = Path(PROMPT_FILE)  # 'prompt.txt' по-умолчанию
    if not path.is_file():
        raise FileNotFoundError(f"Файл {PROMPT_FILE} не найден")

    # читаем файл целиком → одна строка
    PROMPTS = [path.read_text(encoding="utf-8").strip()]
else:
    PROMPTS = [
        "write a quicksort function in Python",
        "The future of artificial intelligence depends on",
    ]


# (изменено) — строим список модификаций без summary; добавим её условно ниже
MODS = [
    ("original", lambda p: p),
#    ("typo first e", lambda p: p.replace("e", "3", 1)),
#    ("add salutation", lambda p: "Dear user, " + p),
    ("префикс ======", lambda p: "="*10 + p),
#    ("префикс вопрос", lambda p: "I have a question. " + p),
#    ("суффикс 10 лет?", lambda p: p + " in the next decade?"),
]

if SUMMARY_ENABLED and _summarizer is not None:
    MODS.append(
        ("summary", lambda p: CUSTOM_WORDS + _summarizer(p, max_length=60, min_length=10, do_sample=False)[0]['summary_text'])
    )


In [6]:
PROMPTS


['write a quicksort function in Python',
 'The future of artificial intelligence depends on']

In [7]:

title = "Печатаем для каждого промпта сам промпт и все его модификации"

print("\n" + "=" * 80)                       # верхняя рамка
print(title.center(80))                     # центрируем на 80 символов
print("=" * 80 + "\n")     
# печатаем все ориганилы промптов и для них все их модификации, промпт за промптом
for i, base_prompt in enumerate(PROMPTS, 1):
    # 🔹 Печатаем исходный текст промпта (можно обрезать при необходимости)
    print(f"🔹 PROMPT #{i}")
    print(f"📝 {base_prompt}")
    print()

    # 🔹 Перебираем *все* модификации, кроме 'original'
    for mod_name, mod_fn in MODS:
        if mod_name == "original":
            continue  # исходник уже выше

        mod_text = mod_fn(base_prompt)

        # Убираем служебный префикс, если это summary
        if mod_name == "summary":
            mod_text = mod_text[len(CUSTOM_WORDS):]

        # Красивый вывод модификации
        print(f"   ➡️ [{mod_name}] {mod_text}")






         Печатаем для каждого промпта сам промпт и все его модификации          

🔹 PROMPT #1
📝 write a quicksort function in Python

   ➡️ [префикс ======] ==========write a quicksort function in Python
🔹 PROMPT #2
📝 The future of artificial intelligence depends on

   ➡️ [префикс ======] ==========The future of artificial intelligence depends on


In [8]:
if USE_FILE_PROMPTS == False:
    TRUNC = 60
else:
    TRUNC = 100   
TOP_K  = 5       # сколько токенов берём на шаг
records = []                              # ← единый список словарей

for prompt in PROMPTS:                    # внешний цикл по промптам
    for mod_name, mod_fn in MODS:         # цикл по модификациям
        mod_prompt = mod_fn(prompt)
        dists      = _multi_step(mod_prompt, STEPS)

        for step_idx, probs in enumerate(dists, start=1):
            topk = torch.topk(probs, TOP_K)

            # перебираем одновременно id токенов и их вероятности
            for rank, (tok_id, prob) in enumerate(
                    zip(topk.indices.tolist(),
                        topk.values.tolist()), start=1):

                records.append({
                    "Prompt": prompt[:TRUNC] + ('…' if len(prompt) > TRUNC else ''),
                    "Step":   step_idx,
                    "Mod":    mod_name,
                    "Rank":   rank,
                    "Token":  _tokenizer.decode([tok_id]),
                    "Prob":   float(prob),
                })

# ─── диагностика ───
for i, rec in enumerate(records):
    if not isinstance(rec, dict):
        print(f"❌ records[{i}] имеет тип {type(rec).__name__}: {rec}")
        break
else:
    print("✅ Все элементы records — dict")

# создаём DataFrame
topk_df = pd.DataFrame(records)
print(topk_df.head())
print(topk_df.columns)


✅ Все элементы records — dict
                                 Prompt  Step       Mod  Rank  Token      Prob
0  write a quicksort function in Python     1  original     1   that  0.293942
1  write a quicksort function in Python     1  original     2     \n  0.092164
2  write a quicksort function in Python     1  original     3      .  0.055051
3  write a quicksort function in Python     1  original     4   \n\n  0.055033
4  write a quicksort function in Python     1  original     5      ,  0.048819
Index(['Prompt', 'Step', 'Mod', 'Rank', 'Token', 'Prob'], dtype='object')


In [9]:

# ---------- A. Сбор топ‑K для каждого prompt × mod × step ----------
# ──────────── А. Подготовка ────────────
for prompt in PROMPTS:
    for mod_name, mod_fn in MODS:
        mod_prompt = mod_fn(prompt)
        dists = _multi_step(mod_prompt, STEPS)

        best_tokens = []                # здесь будем копить «лучшие» токены

        for step_idx, probs in enumerate(dists, start=1):
            topk = torch.topk(probs, TOP_K)

            # — (1) подробные записи, если они всё ещё нужны —
            for rank, (tok_id, prob) in enumerate(zip(topk.indices, topk.values), start=1):
                records.append({ ... })  # оставляю как было

            # — (2) берём самый вероятный токен для текста —
            best_tok = _tokenizer.decode([ topk.indices[0] ])  # rank-1
            best_tokens.append(best_tok)

        # ──────────── Б. Получилось нужное количество шагов ────────────
        best_text = ''.join(best_tokens)      # или ' '.join(...) при необходимости

        # Печатаем сразу:
        short = prompt[:TRUNC] + ('…' if len(prompt) > TRUNC else '')
        print(f"\n=== Prompt: «{short}» | Mod: {mod_name} ===")
        print(best_text)


 


=== Prompt: «write a quicksort function in Python» | Mod: original ===
 that takes a list of integers as input and returns the sorted list. The function should have a time complexity of O(n log n) and should not use any built-in sorting functions. The function should also have a space complexity of O(n) and

=== Prompt: «write a quicksort function in Python» | Mod: префикс ====== ===
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for

=== Prompt: «The future of artificial intelligence depends on» | Mod: original ===
 the ability of humans to develop new technologies that can help us solve the world's most pressing problems. One such technology is the development of AI-powered robots that can assist in the production of renewable energy. These robots can be used to collect and process renewable

=== Prompt: «The future of artificial intelligence depends on» | Mod: префикс ====== ===
 t

In [10]:

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)


# ──────────── тe же DataFrame, плюс новый ────────────


expected = {'Prompt', 'Step', 'Mod', 'Rank', 'Token', 'Prob'}
missing  = expected - set(topk_df.columns)
assert not missing, f"Нет колонок: {missing}"


# длинный формат, как у вас
long_df  = topk_df.melt(
    id_vars    = ["Prompt", "Step", "Rank", "Mod"],
    value_vars = ["Token", "Prob"],
    var_name   = "Metric",
    value_name = "Val"
)


# --- C. Pivot -------------------------------------------
pivot_df = long_df.pivot_table(
    index   = ["Prompt", "Step", "Rank"],
    columns = ["Mod", "Metric"],
    values  = "Val",
    aggfunc = "first",
    sort    = False        # ← сохраняем порядок Token, Prob
)

# 1️⃣ Сортируем ТОЛЬКО строки → (Prompt → Step → Rank)
pivot_df = pivot_df.sort_index()

# 2️⃣ Доп. гарантия: явная перестановка колонок внутри MultiIndex
from itertools import chain
mods = [mod for mod in pivot_df.columns.levels[0]]          # все модификации в порядке level‑0
ordered_pairs = list(chain.from_iterable(
    ((m, "Token"), (m, "Prob")) for m in mods
))

pivot_df = pivot_df[ordered_pairs]  # ← теперь точно Token, Prob, Token, Prob …

# ---------- D. Вывод по каждому промпту отдельно ----------
for prompt_text, grp in pivot_df.groupby(level=0):      # группируем по Prompt
    print("🔹 PROMPT:\n", prompt_text, "\n")
    display(grp.droplevel(0))                           # оставляем Step, Rank
    print("-"*100 + "\n")


# In[11]:




🔹 PROMPT:
 The future of artificial intelligence depends on 



Mod               original              префикс ======          
Metric               Token      Prob             Token      Prob
Step Rank                                                       
1    1                 the  0.220675               the   0.18411
     2                 how  0.104061               how  0.069739
     3                 its  0.042343                 a  0.042836
     4                 our  0.040421               its  0.023548
     5                   a  0.040402               our  0.023389
2    1             ability  0.110883           ability  0.098567
     2         development  0.032065       development  0.037428
     3                 way  0.024823            future  0.034306
     4             success  0.017974      availability  0.023693
     5        availability  0.017271               way  0.019967
3    1                  of  0.522093                to  0.507591
     2                  to   0.46373                of  0.465642
     3                 for  0.003183               for  0.005721
     4                 and  0.002673               and  0.003086
     5                 not  0.000847                \n  0.001585
4    1              humans  0.102838            create  0.067369
     2                  AI  0.092548        understand  0.063625
     3            machines  0.075589             learn  0.031071
     4             machine  0.058537              make  0.030574
     5                 the  0.053272           develop  0.029949
5    1                  to  0.893953                 a  0.081128
     2                 and   0.07689       intelligent  0.060353
     3                   ,  0.008798               and  0.058684
     4                   .  0.001919          machines  0.031784
     5                  in  0.001707               new  0.027138
6    1             develop  0.069008            system  0.052247
     2              create  0.049741              more  0.032398
     3               learn  0.044518             model  0.032242
     4             harness  0.044455               new  0.028161
     5               adapt  0.044032          language  0.027678
7    1                 new  0.106674              that  0.744674
     2                 and  0.091167           capable   0.06808
     3                   a  0.087729              with  0.029807
     4                 the  0.067843                of  0.025926
     5                  an   0.02603             which  0.011553
8    1        technologies  0.100691               can  0.396897
     2                ways  0.083405                is  0.195894
     3                 and  0.061309            learns  0.028447
     4              skills  0.059484              will  0.021606
     5               tools  0.054399               has  0.015403
9    1                that  0.361779             learn  0.133843
     2                 and  0.173243        understand  0.079801
     3                  to  0.105855                be  0.056543
     4                   .  0.079142              make  0.025434
     5                   ,  0.070739            reason  0.020522
10   1                 can  0.319829              from   0.28392
     2                will  0.132625               and  0.238092
     3                 are  0.079859                 ,  0.118056
     4               allow  0.036946           without  0.060424
     5              enable  0.028064                to  0.039529
11   1                help  0.099178              data  0.226438
     2                  be  0.053817        experience  0.125081
     3              better  0.035617               its  0.118553
     4             improve  0.030335               and  0.075146
     5                make  0.030302               the  0.071304
12   1                  us  0.319829               and   0.41533
     2                them  0.230864                 ,  0.152279
     3               solve  0.036961           without  0.109708
     4              people  0.03127

----------------------------------------------------------------------------------------------------

🔹 PROMPT:
 write a quicksort function in Python 



Mod             original              префикс ======          
Metric             Token      Prob             Token      Prob
Step Rank                                                     
1    1              that  0.293942      ==========\n  0.096563
     2                \n  0.092164  ================  0.077588
     3                 .  0.055051       ===========  0.072209
     4              \n\n  0.055033     ===========\n  0.054733
     5                 ,  0.048819        ==========  0.050665
2    1             takes  0.364026               def  0.362687
     2             sorts  0.254923                 #  0.148835
     3              uses  0.038306            import   0.05811
     4           accepts  0.037878              from   0.04878
     5              will  0.033828             """\n  0.030551
3    1                 a  0.413615             quick  0.870327
     2                in  0.189543                 q  0.032021
     3                an  0.156349              sort  0.012197
     4               two  0.086796             Quick  0.010742
     5                as  0.071851         partition  0.009416
4    1              list  0.883492              sort  0.637598
     2            sorted  0.021095             _sort  0.251419
     3          sequence  0.007538              Sort  0.102863
     4                un  0.006657                 s  0.000564
     5            single  0.005657           _select  0.000496
5    1                of  0.709487              (arr  0.302231
     2                as  0.188433            (array  0.201234
     3               and  0.051103              (lst  0.072173
     4                 ,  0.005404             (list  0.036252
     5                 (  0.005306             (data  0.033582
6    1          integers  0.481473              ):\n  0.652343
     2           numbers  0.390919                ):  0.202321
     3          elements  0.023584                 ,  0.083221
     4             items   0.00829                 :  0.015695
     5           strings  0.007994            ):\n\n  0.013884
7    1                as  0.788402                    0.902897
     2               and  0.167933                    0.021622
     3                 ,  0.008868              \tif  0.010333
     4                 (  0.004596                \n  0.007669
     5                or  0.004152                    0.006592
8    1             input  0.783293                if  0.777235
     2                an  0.107307                 #  0.035659
     3               its  0.054614                 n  0.035442
     4                 a  0.033399               def  0.026797
     5          argument  0.006494               """  0.022746
9    1               and  0.760869               len  0.988134
     2                 .  0.093073              (len   0.00621
     3                 ,  0.074156               not  0.001674
     4                \n  0.021433                 (  0.001599
     5               .\n  0.010878               arr  0.001064
10   1           returns  0.623526              (arr  0.999021
     2             sorts  0.343413                 (  0.000682
     3           outputs  0.007229                 (  0.000083
     4             rearr  0.003302            (array  0.000047
     5            prints  0.002893               (\n  0.000026
11   1               the   0.52227                 )  0.935808
     2                 a  0.425739               )<=  0.043991
     3                it  0.028155                )<  0.007902
     4               its  0.008701               )==  0.006453
     5              that  0.004314                )>  0.004945
12   1            sorted  0.719058                <=  0.872229
     2              list  0.196179                 <  0.086227
     3              same  0.049216                 >  0.028218
     4          elements  0.003812                ==  0.012553
     5             array   0.00215                     0.00028
13   1              list  0.975

----------------------------------------------------------------------------------------------------



In [11]:
TRUNC = 60
dist_records = []          # ← сюда попадут KL и Cos для merge

for prompt in PROMPTS:                           # базовый prompt
    base_dists = _multi_step(prompt, STEPS)      # список из STEPS тензоров

    for mod_name, mod_fn in MODS:                # перебираем модификации
        if mod_name == "original":               # «original» оставляем как ref
            continue

        mod_prompt = mod_fn(prompt)
        mod_dists  = _multi_step(mod_prompt, STEPS)

        for step_idx, (p_base, p_mod) in enumerate(zip(base_dists, mod_dists), start=1):
            dist_records.append({
                "Prompt": prompt[:TRUNC] + ('…' if len(prompt) > TRUNC else ''),
                "Step":   step_idx,
                "Mod":    mod_name,
                "KL":     _kl (p_base, p_mod),
                "Cos":    _cos(p_base, p_mod),
            })
dist_df = pd.DataFrame(dist_records)
#print(dist_df.head())

In [12]:
# In[13]:


# (1) глобальная настройка вывода -----------------
pd.set_option("display.max_columns", None)   # показывать все колонки
# ----- 1. Cosine ------------------------------------------------
cos_df = dist_df.pivot_table(
    index   = ["Prompt", "Step"],     # мульти‑индекс: промпт → шаг
    columns = "Mod",                  # колонки — модификации
    values  = "Cos"                   # берём только Cos
).sort_index()




In [13]:
# ----- 2. KL ----------------------------------------------------
kl_df = dist_df.pivot_table(
    index   = ["Prompt", "Step"],
    columns = "Mod",
    values  = "KL"
).sort_index()


In [14]:
# ----- 3. Печать ------------------------------------------------


print("◾️ Cosine similarity")
display(cos_df)

print("\n◾️ KL divergence")
display(kl_df)


# In[14]:



◾️ Cosine similarity


Mod                                                    префикс ======
Prompt                                           Step                
The future of artificial intelligence depends on 1       9.832327e-01
                                                 2       9.694961e-01
                                                 3       9.947702e-01
                                                 4       4.591209e-03
                                                 5       3.581264e-02
                                                 6       2.355575e-02
                                                 7       5.219880e-03
                                                 8       2.988299e-03
                                                 9       5.283432e-03
                                                 10      1.475611e-03
                                                 11      1.324199e-03
                                                 12      1.229828e-02
                                                 13      4.142893e-01
                                                 14      3.249886e-03
                                                 15      1.309145e-04
                                                 16      8.272022e-06
                                                 17      1.806495e-05
                                                 18      3.718287e-05
                                                 19      1.092073e-05
                                                 20      2.223101e-03
                                                 21      1.383576e-04
                                                 22      2.620926e-04
                                                 23      2.718016e-06
                                                 24      4.176333e-03
                                                 25      3.689613e-03
                                                 26      1.801152e-05
                                                 27      3.071025e-05
                                                 28      9.091460e-03
                                                 29      2.584653e-04
                                                 30      3.896619e-03
                                                 31      6.267193e-05
                                                 32      2.513238e-07
                                                 33      4.652428e-07
                                                 34      8.987974e-05
                                                 35      3.607345e-05
                                                 36      2.104898e-05
                                                 37      1.247611e-06
                                                 38      2.237782e-05
                                                 39      1.744768e-06
                                                 40      4.180963e-04
                                                 41      4.367901e-06
                                                 42      9.378906e-05
                                                 43      2.557927e-05
                                                 44      3.604584e-06
                                                 45      4.322740e-07
                                                 46      2.227774e-02
                                                 47      1.175830e-02
                                                 48      2.388474e-05
                                                 49      4.215700e-06
                                                 50      6.995188e-08
write a quicksort function in Python             1       2.308981e-01
                                                 2       1.317462e-05
                                                 3       1.556038e-04
                                                 4       1.396274e-07
                                                 5       1.259972e-04
          


◾️ KL divergence


Mod                                                    префикс ======
Prompt                                           Step                
The future of artificial intelligence depends on 1           0.119426
                                                 2           0.067063
                                                 3           0.013519
                                                 4           7.167924
                                                 5           8.810108
                                                 6           7.617817
                                                 7           6.268603
                                                 8           8.006652
                                                 9           7.283546
                                                 10          9.315948
                                                 11          8.675831
                                                 12         10.951769
                                                 13          2.539988
                                                 14          8.209872
                                                 15          9.217233
                                                 16         15.736468
                                                 17         17.886448
                                                 18         17.114046
                                                 19         14.376358
                                                 20          9.038613
                                                 21         11.081539
                                                 22          9.836124
                                                 23         14.630852
                                                 24          6.803480
                                                 25         15.127703
                                                 26         13.458632
                                                 27         11.796726
                                                 28          9.073189
                                                 29         11.698363
                                                 30         10.449282
                                                 31          9.479165
                                                 32         15.643612
                                                 33         20.578638
                                                 34         15.825772
                                                 35         19.361244
                                                 36         14.223502
                                                 37         15.089619
                                                 38         17.619238
                                                 39         14.981246
                                                 40         12.642393
                                                 41         13.946650
                                                 42         13.855437
                                                 43         12.773559
                                                 44         16.947510
                                                 45         17.244370
                                                 46          3.988279
                                                 47         10.736458
                                                 48         16.123129
                                                 49         22.884560
                                                 50         20.950809
write a quicksort function in Python             1           2.135731
                                                 2          13.039929
                                                 3           8.342905
                                                 4          18.114214
                                                 5          15.977084
          

In [15]:

# --- TOP‑5 по шагам в компактном виде (то есть представление ячейки 7 в более компактном виде)
# не очень понятно, нужно ли, выше то же самое более детально
topk_dfs = []

for prompt, g in topk_df.groupby("Prompt"):
    # g: строки для одного промпта
    # соберём список 5 токенов строкой "tok1, tok2, …"
    g_sorted = g.sort_values(["Step", "Mod", "Rank"])
    g_sorted["TokList"] = g_sorted.groupby(["Step", "Mod"]) ["Token"]  .transform(lambda s: ", ".join(s))
    tidy = (g_sorted.drop_duplicates(subset=["Step", "Mod"])
                    .pivot(index="Step", columns="Mod", values="TokList"))
    tidy.index = [f"Step\u00A0{i}" for i in tidy.index]
    topk_dfs.append((prompt, tidy))

# Печатаем

show_prompt=100 #показываем, сколько символов из промпта мы выводим перед таблицей

for i, (prompt, tidy) in enumerate(topk_dfs, 1):
    prev = prompt.replace("\n", " ")[:show_prompt] + ("…" if len(prompt) > show_prompt else "")
    print(f"\n\n📌 PROMPT {i}: {prev}")
    display(tidy)


# In[ ]:
t2_itogo = tm.time()
print("при количестве шагов", STEPS ,'до этого места прошло', round(t2_itogo - t1_itogo)//60,'минут', 
      round(t2_itogo - t1_itogo)%60,'секунд')



📌 PROMPT 1: The future of artificial intelligence depends on


Mod,original,префикс ======
Step 1,"the, how, its, our, a","the, how, a, its, our"
Step 2,"ability, development, way, success, availability","ability, development, future, availability, way"
Step 3,"of, to, for, and, not","to, of, for, and, \n"
Step 4,"humans, AI, machines, machine, the","create, understand, learn, make, develop"
Step 5,"to, and, ,, ., in","a, intelligent, and, machines, new"
Step 6,"develop, create, learn, harness, adapt","system, more, model, new, language"
Step 7,"new, and, a, the, an","that, capable, with, of, which"
Step 8,"technologies, ways, and, skills, tools","can, is, learns, will, has"
Step 9,"that, and, to, ., ,","learn, understand, be, make, reason"
Step 10,"can, will, are, allow, enable","from, and, ,, without, to"




📌 PROMPT 2: write a quicksort function in Python


Mod,original,префикс ======
Step 1,"that, \n, ., \n\n, ,","==========\n, ================, ===========, ===========\n, =========="
Step 2,"takes, sorts, uses, accepts, will","def, #, import, from, """"""\n"
Step 3,"a, in, an, two, as","quick, q, sort, Quick, partition"
Step 4,"list, sorted, sequence, un, single","sort, _sort, Sort, s, _select"
Step 5,"of, as, and, ,, (","(arr, (array, (lst, (list, (data"
Step 6,"integers, numbers, elements, items, strings","):\n, ):, ,, :, ):\n\n"
Step 7,"as, and, ,, (, or",", , \tif, \n,"
Step 8,"input, an, its, a, argument","if, #, n, def, """""""
Step 9,"and, ., ,, \n, .\n","len, (len, not, (, arr"
Step 10,"returns, sorts, outputs, rearr, prints","(arr, (, (, (array, (\n"


при количестве шагов 50 до этого места прошло 4 минут 23 секунд
